# Important Information

In [ ]:
# This lab notebook simulates the tidal migration of a submoon orbiting a moon, orbiting a planet, orbiting a star.
# The results are recorded as the lifetime of the submoon and saved with the given parameters.
# Parameters below in base SI units, with k2 and Q from Murray-Dermott, densities from Pogge lecture:

# Star, planet, moon, submoon
k2_max = [0.3,0.3,0.3,0.3]
k2_min = [0,0,0,0]
Q_max = [100,100,100,100]
Q_min = [0,0,0,0]
alpha_max = [0.4,0.4,0.4,0.4]
alpha_min = [0,0,0,0]
m_max = [20*1.9885*(10**30),1.9885*(10**30)*0.075,1.9885*(10**30)*0.075,1.9885*(10**30)*0.075]
m_min = [1.9885*(10**30)*0.075,0,0,0]
p_max = [100,17,17,17]
p_min = [(10**-9),0.5,0.5,0.5]

# maximum rate of change of rotational speed per timestep
dOdt_target = 2*(10**-5)

# maximum rate of change of semi-major axis per timestep
dadt_target = 1

# gravitational constant
G = 6.6743015 * (10**-11)

# Import Libraries

In [ ]:
import numpy as np
import random as r
import pandas as pd
import os.path as path
import time

# Define Basic Functions

In [ ]:
# find orbital speed in radians from semi-major axis
def n(a,mp,ms,ns):
    return ns*np.sqrt(G*(mp+ms)/(a**3))

In [ ]:
# find semi-major axis from orbital speed in radians
def semimajor(n,mp,ms):
    return (G*(mp+ms)/(n**2))**(1/3)

In [ ]:
# find the hill radius
def RH(a,mp,ms):
    return a*((ms/(3*mp))**(1/3))

In [ ]:
# find the hill radius factor for greatest influence
def f(Op,ns):
    return Op/np.abs(Op)*ns*0.07 + 0.43

In [ ]:
# find constant for change in semi-major axis over time
def b(k,ms,mp,Cp,a):
    return (3*k*ms*((Cp/a)**5)*a)/mp

In [ ]:
# find constant for change in rotational speed over time
def d(k,alpha,ms,mp,Cp,a):
    return ((3*k*(ms**2))*((Cp/a)**3)*(n(a,mp,ms,1)**2))/(alpha*(mp*(mp + ms)))

In [ ]:
# find the total satellite mass for an orbit
def satMass(i):
    mVal = 0
    for j in range(i+1,4):
        mVal=mVal+m[j]
    return mVal

In [ ]:
# find change in semi-major axis over time for any index
def dadt(i):
    nVal = n(a[i],m[i],satMass(i),ns[i])
    if i==0:
        return nVal*((O[i]-nVal)*b(k[i],satMass(i),m[i],C[i],a[i])+(O[i+1]-nVal)*b(k[i+1],m[i],m[i+1],C[i+1],a[i]))
    return nVal*((O[i]-nVal)*b(k[i],satMass(i),m[i],C[i],a[i])+(O[i+1]-nVal)*b(k[i+1],m[i],m[i+1],C[i+1],a[i]))+2*unbalL(i-1)*((m[i]+satMass(i))/(m[i]*satMass(i)))/(a[i]*nVal)*(O[i]-n(a[i-1],m[i],m[i-1],ns[i-1]))

In [ ]:
# find change in rotational speed for any index
def dOdt(i):
    sum = 0
    if i<3:
        sum=sum-(O[i]-n(a[i],m[i],satMass(i),ns[i]))*d(k[i],alpha[i],satMass(i),m[i],C[i],a[i])
    if i>0:
        sum=sum-(O[i]-n(a[i-1],m[i],m[i-1],ns[i-1]))*d(k[i],alpha[i],m[i-1],m[i],C[i],a[i-1])
    return sum

In [ ]:
# find unbalanced angular momentum term
def unbalL(i):
    return 1.5*k[i+1]*((C[i+1]/a[i])**5)*(n(a[i],m[i],satMass(i),ns[i])**2)*(a[i]**2)*(m[i]**2)*((1/(m[i]+m[i+1]))-(satMass(i)/(m[i+1]*(m[i]+satMass(i)))))

# Define Functions for Evolving Timescale

In [ ]:
# initialize constants
O = [0,0,0,0]
O_init = [0,0,0,0]
a = [0,0,0]
a_init = [0,0,0]

ns = [0,0,0]
n_init = [0,0,0]
m = [0,0,0,0]
k = [0,0,0,0]
alpha = [0,0,0,0]
C = [0,0,0,0]

In [ ]:
# Randomize parameter space
def randomize():
    found = False
    while(found==False):
        for i in range(0,4):
            m[i]=m_min[i]+r.random()*(m_max[i]-m_min[i])
            p=p_min[i]+r.random()*(p_max[i]-p_min[i])
            C[i]=(3*m[i]/(4*np.pi*p))**(1/3)
            O[i]=0.2*(r.random()-0.5)*n(C[i],m[i],0,1)
            O_init[i]=O[i]
            alpha[i]=alpha_min[i]+r.random()*(alpha_max[i]-alpha_min[i])
        for i in range(0,3):
            nVal=0
            nSign=1
            if r.random()>0.5:
                nSign=-1
            if i==0:
                nVal=n(C[i]+r.random()*(1.975*(10**13)-C[i]),m[i],satMass(i),nSign)
            else:
                nVal=n(C[i]+r.random()*(RH(a[i-1],m[i-1],m[i])-C[i])*f(O[i],nSign),m[i],satMass(i),nSign)
            a[i]=semimajor(nVal,m[i],satMass(i))
            ns[i]=nSign
            a_init[i]=a[i]
            n_init[i]=nVal
        for i in range(0,4):
            Q=Q_min[i]+r.random()*(Q_max[i]-Q_min[i])
            k2=k2_min[i]+r.random()*(k2_max[i]-k2_min[i])
            if i<3:
                k[i]=k[i]+k2/(Q*np.abs(O_init[i]-n_init[i]))
            if i>0:
                k[i]=k[i]+k2/(Q*np.abs(O_init[i]-n_init[i-1]))
            if i>0 & i<3:
                k[i]=k[i]/2
        # Cull impossible scenarios
        found = True
        for i in range(0,3):
            if satMass(i)>m[i] or C[i]>a[i]:
                found = False

In [ ]:
# set timestep to 1y, set maximum timestep to 10Gy, the timestep ceiling to 100ky, and the timestep floor to 1y
timestep = 1*60*60*24*365.24
timestepceil = 100000*60*60*24*365.24
timestepfloor = 1*60*60*24*365.24
maxTimeStep = 1*60*60*24*365.24*(10**9)

# set a time limit for calculation
timeLimit = 10

# tick through each timestep, return lifetime or system, return -1 if time limit reached
def lifetime(timestep):
    life = 0
    startTime = time.time()
    while(life<maxTimeStep):
        if time.time()>startTime+timeLimit:
            return -1
        maxdOdt = 0
        maxdadt = 0
        for i in range(0,4):
            change = dOdt(i)
            O[i]=O[i]+change*timestep
            if np.abs(change)>maxdOdt:
                maxdOdt = np.abs(change)
        for i in range(0,3):
            change = dadt(i)
            a[i]=a[i]+change*timestep
            if np.abs(change)>maxdadt:
                maxdadt = np.abs(change)
        # check for escapes
        for i in range(1,3):
            if RH(a[i-1],m[i-1],m[i])*f(O[i],ns[i])<a[i]:
                return life
        # check for Roche Limit encroaches
        for i in range(1,4):
            if RH(a[i-1],m[i-1],m[i])<C[i]:
                return life
        # check for object crashes
        for i in range(0,3):
            if a[i]<C[i]:
                return life
        # check for odd orbital overlaps
        if a[1]>a[0]+C[0] or a[2]>a[1]+C[1] or a[0]<a[1]+a[2]+C[0]:
            return life
        # add lifetime
        life=life+timestep
        # adjust timestep based on change, WIP
        timestep=np.maximum(np.minimum(timestep*np.minimum(dOdt_target/maxdOdt,dadt_target/maxdadt),timestepceil),timestepfloor)
    return maxTimeStep

In [ ]:
# enumerate strings for .csv appending
def csvHeader():
    theString = ""
    for i in range(0,4):
        theString=theString+"k_"+str(i)+","
        theString=theString+"O_init_"+str(i)+","
        theString=theString+"m_"+str(i)+","
        theString=theString+"alpha_"+str(i)+","
        theString=theString+"C_"+str(i)+","
    for i in range(0,3):
        theString=theString+"a_init_"+str(i)+","
        theString=theString+"n_init_"+str(i)+","
    theString=theString+"Lifetime (y)\n"
    return theString
def csvString(life):
    theString = ""
    for i in range(0,4):
        theString=theString+str(k[i])+","
        theString=theString+str(O_init[i])+","
        theString=theString+str(m[i])+","
        theString=theString+str(alpha[i])+","
        theString=theString+str(C[i])+","
    for i in range(0,3):
        theString=theString+str(a_init[i])+","
        theString=theString+str(n_init[i])+","
    theString=theString+str(life/(365.24*24*60*60))+"\n"
    return theString

In [ ]:
# solve for and save many random points
numPoints = 100*60*24*3
t = time.localtime()
fileName = "numerical out - " + str(t[0]) + "-" + str(t[1]) + "-" + str(t[2]) + "-" + str(t[3]) + "-" + str(t[4]) + "-" + str(t[5]) + ".csv"
file = open(fileName, "w")
file.write(csvHeader())
startTime = time.time()
for i in range(0, numPoints):
    randomize()
    life = lifetime(timestep)
    file.write(csvString(life))
    if life>0:
        print(str(life/(365.24*24*60*60)) + " years")
    file.flush()
file.close()
print("Time taken for " + str(numPoints) + " data points: " + str(time.time()-startTime) + "s, averaging " + str((time.time()-startTime)/numPoints) + "s per data point.")